# Import Libraries

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision #library to deal with computer vision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as opt

# Download MINIST train and test dataset from Torchvision 

In [ ]:
train_data = torchvision.datasets.MNIST(root='data', train=True,
                                   download=True, transform=transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root='data', train=False,
                                  download=False, transform=transforms.ToTensor())

# Load data in dataloaders to process the data in batches

In [ ]:
trainloader=torch.utils.data.DataLoader(train_data,batch_size=4,shuffle=True)

In [ ]:
dataiter=iter(trainloader)

#every next() will give an instance of 4 images and its labels
images,labels=dataiter.next()

#no.of.images,RGB(size),image dim
print(images.shape)
#class(batchsize)
print(labels.shape)
#RGB(size),image dim
print(images[0].shape)
#class of 1st image
print(labels[0].item())

In [ ]:
img=images[0]
#convert tensor to numpy
npimg=img.numpy()

#plot it using imshow
plt.figure(figsize=(1,1))
plt.imshow(npimg.reshape(28,28))
plt.show()

# class creation with Forward Layers

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.classifier=nn.Sequential(nn.Linear(784,48),
                                  nn.BatchNorm1d(48),
                                  nn.Tanh(),
                                  nn.Linear(48,24),
                                  nn.BatchNorm1d(24),
                                  nn.Tanh(),
                                  nn.Linear(24,10))
  def forward(self,x):
    x=x.view(x.size(0),-1)
    return self.classifier(x)

    


In [ ]:
trainloader=torch.utils.data.DataLoader(train_data,batch_size=512,shuffle=True)
testloader=torch.utils.data.DataLoader(test_data,batch_size=512,shuffle=False)

# Moving the Data to GPU

In [ ]:
# In google colab Edit->Notebook settings->HardwareAccelerator->GPU

device=torch.device('cuda:0')
model=Net().to(device)
fn=opt.SGD(model.parameters(),lr=0.01)
loss_fn=nn.CrossEntropyLoss()



In [ ]:
def evaluate(dataloader):
  total,correct=0,0
  for loader in dataloader:
    images,labels=loader
    images=images.to(device)
    labels=labels.to(device)
    output=model(images)
    _,pred=torch.max(output.data,1)
    total+=labels.size(0)
    correct+=(pred==labels).sum().item()
  return 100*correct/total

# Training 

In [ ]:
epochs=10

def train(model,loss_fn,Optimizer,device):
  lossarr=[]
  for epoch in range(epochs):
    for loader in trainloader:
      Optimizer.zero_grad()

      images,labels=loader
      images=images.to(device)
      labels=labels.to(device)

   
      outputs=model(images)
      loss=loss_fn(outputs,labels)
      loss.backward()
      Optimizer.step()
    
    lossarr.append(loss)
    print('epoch:%d,Train accuracy:%0.2f,Test accuracy:%0.2f' %(epoch,evaluate(trainloader),evaluate(testloader)))

  plt.plot(lossarr)
  plt.show()

In [ ]:
train(model,loss_fn,fn,device)